# DATA TRANSFORMATION - 11_Ante_Fam
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'Data/'

## 1. Load

In [2]:
modulo = 'Antecedentes_familiares'

In [3]:
dcc = gf.diccionario_llaves()

In [4]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,EPS,fecha,FE_INICIO,FE_FIN,Diagnostico,CodDiagnostico,Parentesco,year,month,year_month
0,1998401,EPS/SURA,2012-07-09,2006-01-01,NaT,"ASMA, NO ESPECIFICADA",J459,MADRE,2012,7,201207
1,535255,EPS/SURA,2011-07-29,NaT,2011-07-29,"DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION...",E149,ABUELO,2011,7,201107
2,1004538,EPS/SURA,2012-01-17,2009-06-10,NaT,HIPERTENSION ESENCIAL (PRIMARIA),I10X,PADRE,2012,1,201201
3,788432,EPS/SURA,2012-01-06,2007-09-27,NaT,HIPERTENSION ESENCIAL (PRIMARIA),I10X,PADRE,2012,1,201201
4,788432,EPS/SURA,2012-01-06,2007-09-27,NaT,HIPERTENSION ESENCIAL (PRIMARIA),I10X,MADRE,2012,1,201201


## 2. Transform

In [5]:
base = gf.letra_codigo( base, 'CodDiagnostico' )
base.head()

,id,EPS,fecha,FE_INICIO,FE_FIN,Diagnostico,CodDiagnostico,Parentesco,year,month,year_month,CodDiagnostico_cod
0,1998401,EPS/SURA,2012-07-09,2006-01-01,NaT,"ASMA, NO ESPECIFICADA",J459,MADRE,2012,7,201207,j
1,535255,EPS/SURA,2011-07-29,NaT,2011-07-29,"DIABETES MELLITUS, NO ESPECIFICADA SIN MENCION...",E149,ABUELO,2011,7,201107,e
2,1004538,EPS/SURA,2012-01-17,2009-06-10,NaT,HIPERTENSION ESENCIAL (PRIMARIA),I10X,PADRE,2012,1,201201,i
3,788432,EPS/SURA,2012-01-06,2007-09-27,NaT,HIPERTENSION ESENCIAL (PRIMARIA),I10X,PADRE,2012,1,201201,i
4,788432,EPS/SURA,2012-01-06,2007-09-27,NaT,HIPERTENSION ESENCIAL (PRIMARIA),I10X,MADRE,2012,1,201201,i


* Number of delivered doses per month
* Number of different diseases per month

In [6]:
base_ = base.groupby(['id','year','month','CodDiagnostico_cod'])["id"].count().\
    reset_index( name = 'antf' )
base_.head()

,id,year,month,CodDiagnostico_cod,antf
0,529410,2011,8,i,1
1,535255,2011,7,e,1
2,557377,2016,2,r,1
3,664340,2015,10,j,2
4,673765,2011,10,j,1


In [7]:
# all diagnoses different form j will be in the same category
base_['diag'] = base_['CodDiagnostico_cod'].apply( lambda x : 'j' if x == 'j' else 'otra' )

In [8]:
# pivot table to have separate variables, nans replaced with zero
base_p = base_.pivot_table(index=['id','year','month'], columns='diag', values=['antf'], aggfunc=np.sum).reset_index()
base_p.columns = ['_'.join(col).strip() for col in base_p.columns.values]
base_p.fillna( 0 , inplace = True)
base_p.head()

,id_,year_,month_,antf_j,antf_otra
0,529410,2011,8,0.0,1.0
1,535255,2011,7,0.0,1.0
2,557377,2016,2,0.0,1.0
3,664340,2015,10,2.0,0.0
4,673765,2011,10,1.0,0.0


In [9]:
base_p.rename( columns={ 'id_':'id', 'year_':'year', 'month_':'month' }, inplace=True)
base_p.head()

,id,year,month,antf_j,antf_otra
0,529410,2011,8,0.0,1.0
1,535255,2011,7,0.0,1.0
2,557377,2016,2,0.0,1.0
3,664340,2015,10,2.0,0.0
4,673765,2011,10,1.0,0.0


## 3. Merge with ids table

In [10]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_antf = ids_mensual.merge( base_p, how='left')
base_final_antf.head()

,id,year,month,antf_j,antf_otra
0,500547,2016,1,NaN,NaN
1,500547,2016,2,NaN,NaN
2,500547,2016,3,NaN,NaN
3,500547,2016,4,NaN,NaN
4,500547,2016,5,NaN,NaN


In [12]:
base_final_antf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53280 entries, 0 to 53279
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         53280 non-null  int64  
 1   year       53280 non-null  int64  
 2   month      53280 non-null  int64  
 3   antf_j     9 non-null      float64
 4   antf_otra  9 non-null      float64
dtypes: float64(2), int64(3)
memory usage: 2.4 MB
